In [1]:
import pandas as pd
import numpy as np
from config import *
from modules.transformations import *

In [2]:
df = pd.read_csv(DATA_FILE_PATH)
dictionary = pd.read_csv(DICTIONARY_PATH)

The first step for my is getting this data into a format that's more easily digestible.
The formatting is initally as an incredibly sparse matrix, but I'm more interested in representing the data as a series of vectors.

In [3]:
df.rename(
    columns = COLUMN_MAPPINGS,
    inplace= True
)

dictionary.rename(
    columns = COLUMN_MAPPINGS,
    inplace= True
)

df.set_index('source', inplace=True)
df

,ABW,AFG,AGO,ALB,ARE,ARG,ARM,ATG,AUS,AUT,...,VCT,VEN,VIR,VNM,VUT,WSM,YEM,ZAF,ZMB,ZWE
source,,,,,,,,,,,,,,,,,,,,,
ABW,0,0,0,0,0,1,0,2,36,2,...,15,337,78,0,0,0,0,0,0,0
AFG,0,0,0,0,501,4,0,0,5859,801,...,0,0,0,0,0,0,0,7,0,0
AGO,0,0,0,0,0,2,0,0,41,21,...,0,19,0,0,0,0,0,1957,21090,0
ALB,0,0,0,0,0,21,1,0,261,212,...,0,3,0,0,0,0,0,22,0,0
ARE,1,233,0,1,0,2,1,1,1157,60,...,0,6,0,83,0,0,6610,99,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WSM,0,0,0,0,0,0,0,0,4742,0,...,0,0,0,0,1,0,0,0,0,0
YEM,0,0,0,0,31178,0,0,0,253,28,...,0,1,0,0,0,0,0,24,0,0
ZAF,0,3,1801,8,16,122,0,5,10042,460,...,0,41,0,14,0,0,7,0,1265,6123


In [4]:
data = matrix_to_vectors(df)

In [5]:
data

array([['ABW', 'ABW', 0],
       ['ABW', 'AFG', 0],
       ['ABW', 'AGO', 0],
       ...,
       ['ZWE', 'ZAF', 276684],
       ['ZWE', 'ZMB', 5573],
       ['ZWE', 'ZWE', 0]], shape=(40000, 3), dtype=object)